In [1]:
import sys, time
sys.path.append('../')

In [2]:
from ipycanvas import canvas
from demo import *
from model import *
from pythreejs import *

In [3]:
c = canvas.Canvas(width=600, height=200, sync_image_data=True)
pipe = AdvancedPipe(Circle(1, 0), Circle(1, 0), 10, canvas=c)

In [4]:
height = 7.5
ref_cyl = CylinderBufferGeometry(pipe.i1.d / 2, pipe.i2.d / 2, height, 16, 8)
cyl = BufferGeometry.from_geometry(ref_cyl)

In [5]:

demo = Demo(pipe.params, pipe, drawable=c)

In [7]:
skinIndices = []
skinWeights = []
vertices = cyl.attributes['position'].array
boneHeight = ref_cyl.height / 2
for i in range(vertices.shape[0]):

    y = vertices[i, 1] + 0.5 * ref_cyl.height

    skinIndex = y // boneHeight
    skinWeight = ( y % boneHeight ) / boneHeight

    # Ease between each bone
    skinIndices.append([skinIndex, skinIndex + 1, 0, 0 ])
    skinWeights.append([1 - skinWeight, skinWeight, 0, 0 ])

cyl.attributes = dict(
    cyl.attributes,
    skinIndex=BufferAttribute(skinIndices),
    skinWeight=BufferAttribute(skinWeights),
)

shoulder = Bone(position=(0, -25, 0))
elbow = Bone(position=(0, 25, 0))
hand = Bone(position=(0, 25, 0))

shoulder.add(elbow)
elbow.add(hand)
bones = [shoulder, elbow, hand]
skeleton = Skeleton(bones)

pipeMesh = SkinnedMesh(cyl, MeshPhongMaterial(side='DoubleSide', skinning=True, color='gray', opacity=0.3, transparent=True), rotation=[pymath.pi, 0,pymath.pi / 2, 'XYZ'], position=[0, 0, -5])
pipeMesh.add(bones[0])
pipeMesh.skeleton = skeleton

In [8]:
water = CylinderBufferGeometry(pipe.i1.d / 4, pipe.i1.d / 4, height / 3, 16, 8, thetaLength=pymath.pi)
waterMesh = Mesh(water, MeshStandardMaterial(color='lightblue', opacity=1, transparent=False), rotation=[pymath.pi, 0, pymath.pi / 2, 'XYZ'], position=[-height / 3, 0, -5])
cam = PerspectiveCamera(position=[0, 0, 0], quaternion=[0, 0, 0, 0], aspect=600/200)
scene = Scene(children=[pipeMesh, waterMesh, camera, AmbientLight(color='#FFFFFF')], background=None)
renderer = Renderer(scene=scene, camera=camera, controls=[OrbitControls(controlling=camera)], width=600, height=200, alpha=True, clearOpacity=0)
demo.show()
renderer

/home/kipoka/anaconda3/envs/hydraulics/lib/python3.10/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Output()

Renderer(camera=PerspectiveCamera(aspect=3.0, children=(DirectionalLight(color='white', intensity=0.5, matrixW…

In [9]:
def update_pipe(args):
    pipeMesh.skeleton.bones[1].rotation = [0, 0, (pipe.i1yParam.real() - pipe.i2yParam.real()) / 100, 'XYZ']
    waterMesh.rotation = [waterMesh.rotation[0], waterMesh.rotation[1], pipeMesh.skeleton.bones[1].rotation[2], 'XYZ']
    g3 = LineSegmentsGeometry(
    positions=[
        [[0, pipe.i1yParam.real() / 10, -5], [100, (pipe.i1yParam.real()) / 10, -5]],
    ],
    colors=[
        [[1, 0, 0], [1, 0, 1]],
    ],
    )
    m3 = LineMaterial(linewidth=10, vertexColors='VertexColors')
    line3 = LineSegments2(g3, m3)
    #scene.add(line3)
    #pipeMesh.skeleton.bones[2].rotation = [0, 0, pipe.i2yParam.real() / 100, 'XYZ']
    cyl = CylinderBufferGeometry(pipe.i1.d / 2, pipe.i2.d / 2, height, 16, 8)
    #cyl_geom = BufferGeometry.from_geometry(cyl)
    #vertices = cylinder.attributes['position'].array
    #vertices[15 // 2] = [0, 0, 0]
    #print(cylGeom.vertices)
    pipeMesh.geometry = cyl
    
    #cv = pipe.get_drawing()
    #scene.add(cv)
    
    #pipeMesh.position = [pipeMesh.position[0], pipe.i1yParam.real(), pipeMesh.position[2]]

In [10]:
pipe.observe(update_pipe)